In [7]:
import os
import json
import time
from collections import deque
import operator
from typing import Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from typing import Optional, TypedDict, List, Dict, Any, Union

from langgraph.constants import Send
from langchain_groq import ChatGroq
from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    SystemMessage,
    BaseMessage
)
from langgraph.graph import StateGraph, START, END

###############################################
# SPECDEC 30k tokens/minute THROTTLING LOGIC
###############################################
MAX_TOKENS_PER_MIN = 30000
token_usage_window_specdec = deque()

def record_specdec_token_usage(token_count):
    now = time.time()
    token_usage_window_specdec.append((now, token_count))
    # Remove any usage older than 60 seconds
    while token_usage_window_specdec and (now - token_usage_window_specdec[0][0]) > 60:
        token_usage_window_specdec.popleft()

def get_specdec_tokens_last_minute():
    now = time.time()
    return sum(t[1] for t in token_usage_window_specdec if (now - t[0]) <= 60)

def approximate_token_count(conversation: List[BaseMessage]) -> int:
    """
    A naive approach to estimate token usage by counting words.
    If total words > 6000, we'll avoid SpecDec (it can handle ~8000 tokens).
    In a production environment, consider a more accurate token counting method.
    """
    total_words = 0
    for msg in conversation:
        total_words += len(msg.content.split())
    return total_words

def safe_invoke_specdec(structured_llm, conversation, **kwargs):
    """
    Invokes SpecDec LLM calls while ensuring we:
     1) Avoid using SpecDec if the input tokens exceed 6000.
     2) Use Versatile if rolling token usage + input + expected output > 28k.
        (Since SpecDec has an ~8k token limit total, and we're accounting for 2k output tokens.)
    """
    # Approximate token usage of the input conversation
    input_tokens_approx = approximate_token_count(conversation)
    
    # Estimate total tokens after the request
    expected_output_tokens = 2000  # Fixed assumption as per logic
    rolling_token_usage = get_specdec_tokens_last_minute()
    estimated_total_tokens = rolling_token_usage + input_tokens_approx + expected_output_tokens

    # Check if we should fallback to modelVers
    if input_tokens_approx > 6000 or estimated_total_tokens > 28000:
        # Fallback to modelVers with the same structured output schema
        fallback_llm = modelVers.with_structured_output(
            structured_llm.output_schema,
            method=structured_llm.method,
            include_raw=structured_llm.include_raw
        )
        output = fallback_llm.invoke(conversation, **kwargs)
        return output

    # Proceed with SpecDec if limits are respected
    output = structured_llm.invoke(conversation, **kwargs)

    # Record the actual tokens used for SpecDec to update rolling token usage
    used_tokens = output.get("response_metadata", {}).get("token_usage", {}).get("total_tokens", 0)
    record_specdec_token_usage(used_tokens)
    return output



os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("CUSTOM_TRACING_V2", "True")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("CUSTOM_ENDPOINT", "https://api.smith.langchain.com")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("CUSTOM_API_KEY", "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")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("CUSTOM_PROJECT", "CFOLytics_reportgenerator")

# Manually set the __file__ variable to the notebook's directory
__file__ = os.path.abspath("notebook_name.ipynb")

def load_xml_instructions(filename: str) -> str:
    """
    Load system instructions from 'XML_instructions/filename' if you keep them externally.
    Otherwise, just inline your prompts as strings.
    """
    current_dir = os.path.dirname(os.path.abspath(__file__))
    file_path = os.path.join(current_dir, "XML_instructions", filename)
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

# Prompts we will use
layout_prompt = load_xml_instructions("render_layout.xml")
component_prompt = load_xml_instructions("component_content_gen.xml")

# LLM
modelVers = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    api_key="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",
    disable_streaming=True
)

modelSpec = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-specdec",
    api_key="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",
    disable_streaming=True
)

class Components(BaseModel):
    Components: list[str]

class ComponentConfig(BaseModel):
    config : dict

class OverallState(TypedDict):
    ReportQuery: str
    POV: list
    ReportMetadata: Annotated[List[Dict[str, Any]], operator.add]
    JsonLayout: dict
    Components: list
    JsonLayoutWithComponentConfig: Annotated[list, operator.add]
    Lists: list
    JsonLists: Annotated[list, operator.add]

class NestedNumberFormat(BaseModel):
    scale: str
    decimals: int

from typing import List, Optional, Union
from pydantic import BaseModel, Field

# Define the lowest-level models first
class GridColumns(BaseModel):
    sm: Union[int, str]
    md: Union[int, str]
    lg: Union[int, str]

    class Config:
        extra = "forbid"

class ColSpan(BaseModel):
    sm: Union[int, str]
    md: Union[int, str]
    lg: Union[int, str]

    class Config:
        extra = "forbid"

class NestedNumberFormat(BaseModel):
    scale: str
    decimals: int

    class Config:
        extra = "forbid"

# Define rows and columns next
class Component(BaseModel):
    id: str
    type: str
    title: Optional[str] = None
    AI_Generation_Description: Optional[str] = Field(
        None,
        alias="AI Generation Description"
    )
    noborder: Optional[bool] = None
    height: Optional[int] = None
    numberFormat: Optional[NestedNumberFormat] = None
    config: Optional["Layout"] = None  # Use forward reference for recursive Layout

    class Config:
        populate_by_name = True
        extra = "forbid"

class Column(BaseModel):
    colSpan: ColSpan
    components: List[Component]

    class Config:
        extra = "forbid"

class Row(BaseModel):
    columns: List[Column]

    class Config:
        extra = "forbid"

# Define layout after rows and columns
class Layout(BaseModel):
    gridColumns: Optional[GridColumns] = None
    rows: List[Row]

    class Config:
        extra = "forbid"

# Define the top-level models
class NumberFormat(BaseModel):
    currency: str
    scale: str
    decimals: int

    class Config:
        extra = "forbid"

class ReportConfig(BaseModel):
    reportTitle: str
    numberFormat: NumberFormat
    layout: Layout

    class Config:
        extra = "forbid"

def generate_layout(state: OverallState):
    system_instructions = load_xml_instructions("render_layout.xml")
    system_msg = SystemMessage(content=system_instructions)
    user_msg = HumanMessage(content=state["ReportQuery"])
    report_metadata = state["ReportMetadata"]

    structured_llm = modelSpec.with_structured_output(
        ReportConfig,
        method="json_mode",
        include_raw=True
    )
    conversation = [system_msg] + [user_msg]

    # Use safe_invoke_specdec to ensure we don't exceed token limits
    output = safe_invoke_specdec(structured_llm, conversation, stream=False, response_format="json")

    if output["parsed"] is None:
        # Construct a meaningful error message
        error_message = (
            "Parsing failed: The 'parsed' field in the output is None.\n"
            "Raw LLM Output:\n"
            f"{output.get('raw', 'No raw output available')}\n\n"
            "Additional Context:\n"
            f"Model Name: {output.get('response_metadata', {}).get('model_name', 'Unknown')}\n"
            f"Token Usage: {output.get('response_metadata', {}).get('token_usage', 'Unknown')}\n"
            f"Finish Reason: {output.get('response_metadata', {}).get('finish_reason', 'Unknown')}\n"
            "Please check the raw output for errors or unexpected formatting."
        )
        raise Exception(error_message)

    parsed_output = output["parsed"].model_dump()

    # Find components
    components = []
    def walk(obj):
        if isinstance(obj, dict):
            if "components" in obj and isinstance(obj["components"], list):
                for c in obj["components"]:
                    components.append(c)
            for v in obj.values():
                walk(v)
        elif isinstance(obj, list):
            for v in obj:
                walk(v)
    walk(parsed_output)
    
    parsed_output["POV"] = state["POV"]

    # Output both JsonLayout and Components
    return {"JsonLayout": parsed_output, "Components": components, "ReportMetadata": report_metadata}


def continue_to_components(state: OverallState):
    return [Send("generate_component", {"component": c}) for c in state["Components"]]

def continue_to_lists(state: OverallState):
    return [Send("generate_list_subchart",{"List": l,"ReportMetadata": state["ReportMetadata"] }) for l in state["Lists"]]

class ComponentState(TypedDict):
    component: dict

def generate_component(state: ComponentState):
    component = state["component"]
    ai_description = component.get("AI_Generation_Description", "None")
    ai_description = ai_description.strip() if isinstance(ai_description, str) else ai_description
    ai_description = None if ai_description == "None" else ai_description

    component_id = component.get("id", "NoId")
    if(ai_description is not None):
        system_instructions = load_xml_instructions("component_content_gen.xml")
        system_msg = SystemMessage(content=system_instructions)
        user_msg = HumanMessage(content=ai_description)
        
        structured_llm = modelSpec.with_structured_output(
            ComponentConfig,
            method="json_mode",
            include_raw=True
        )

        conversation = [system_msg] + [user_msg]
        # This is a Versatile model call, so we keep it as is

        output = safe_invoke_specdec(structured_llm, conversation, stream=False, response_format="json")
        parsed_output = output["parsed"].model_dump()
    else:
        parsed_output = {}

    # Generate configuration
    generated_config = {
        "id": component_id,
        "generatedConfig": parsed_output
    }

    return {"JsonLayoutWithComponentConfig": [generated_config]}  # Return as list

def update_json_layout(state: OverallState):
    components_configs = state["JsonLayoutWithComponentConfig"]
    updated_layout = state["JsonLayout"]

    def walk_and_update(layout, configs):
        if isinstance(layout, dict):
            if "components" in layout and isinstance(layout["components"], list):
                for component in layout["components"]:
                    for config in configs:
                        if config["id"] == component.get("id"):
                            component.update(config["generatedConfig"])
            for v in layout.values():
                walk_and_update(v, configs)
        elif isinstance(layout, list):
            for v in layout:
                walk_and_update(v, configs)

    walk_and_update(updated_layout, components_configs)
    return {"JsonLayout": updated_layout}

def gatheruniquelists(state: OverallState):
    """
    Gathers all unique 'lists' definitions from the components in 'JsonLayoutWithComponentConfig'
    and stores them in the 'Lists' and 'JsonLists' fields of the state.
    """
    new_lists = []
    
    for component_config in state.get("JsonLayoutWithComponentConfig", []):
        generated_config = component_config.get("generatedConfig", {})
        config_obj = generated_config.get("config", {})
        comp_lists = config_obj.get("lists", [])

        # Add only unique list definitions
        for list_def in comp_lists:
            if list_def not in new_lists:
                new_lists.append(list_def)

    state["Lists"] = new_lists

    return {
        "Lists": new_lists
    }

#########################################################
# 1. Extend the TypedDict for your subchart state
#########################################################
class ListSubchartState(TypedDict):
    """
    Extend your state to include any fields needed
    for checking, creating, or returning a list.
    """
    List: dict              # The raw definition of a list or instructions for how to create it
    listExists: bool        # Whether the list already exists
    listType: str           # 'dynamic' or 'fixed'
    dimensions: List        # Top-level dimensions for the list
    ReportMetadata: Annotated[List[Dict[str, Any]], operator.add]
    JsonLists: List[dict]   # The final generated JSON list(s)
    FinalList: dict         # The actual final list data (if you want it separate from JsonLists)


#########################################################
# 2. Define placeholder nodes
#########################################################

def check_if_list_exists(state: ListSubchartState):
    """
    Placeholder: checks if the list already exists.
    Set 'listExists' in the returned partial state.
    """
    # Example logic (replace with real check):
    list_exists_flag = False  # Placeholder: always assume it doesn't exist

    return {"listExists": list_exists_flag}


def return_existing_list(state: ListSubchartState):
    """
    Placeholder: if the list already exists, retrieve and return it.
    """
    # Example logic (replace with real retrieval):
    existing_list_data = {"someExistingList": True}  # Placeholder

    return {
        "FinalList": existing_list_data,
        "JsonLists": [existing_list_data]
    }

class DynamicOrFixedReply(BaseModel):
    type: str
    dimensions: list

def check_dynamic_or_fixed(state: ListSubchartState):
    """
    Uses an LLM (SpecDec) to decide if the list is 'Fixed' or 'Dynamic' and determine
    which top-level dimensions apply to the list.
    """
    current_list = state["List"]
    report_metadata = state["ReportMetadata"]
    top_level_dims = [
        {
            "name": dim["name"],
            "alias": dim["alias"],
            "members": [member["Name"] for member in dim.get("dimensionContent", [])[:3]]
        }
        for dim in report_metadata
    ]
    
    dynamic_or_fixed_prompt = load_xml_instructions("dynamic_or_fixed_prompt.xml")
    system_msg = SystemMessage(content=dynamic_or_fixed_prompt)

    user_input = {
        "listObject": current_list,
        "topLevelDimensions": top_level_dims
    }

    user_msg = HumanMessage(content=json.dumps(user_input, indent=2))
        
    structured_llm = modelSpec.with_structured_output(
        DynamicOrFixedReply,
        method="json_mode",
        include_raw=True
    )
    conversation = [system_msg] + [user_msg]

    # Use safe_invoke_specdec
    output = safe_invoke_specdec(structured_llm, conversation, stream=False)
    parsed_output = output["parsed"].model_dump()

    return {
        "listType": parsed_output["type"],
        "dimensions": parsed_output["dimensions"]
    }

def build_hierarchy_string(filtered_metadata, parent_id=None, indent=0):
    """
    Convert multiple dimensions of `filtered_metadata` into a hierarchical string representation.
    """
    result = ""
    for metadata in filtered_metadata:
        dimension_content = metadata.get("dimensionContent", [])
        for item in dimension_content:
            item_parent_id = item.get("ParentID")
            if item_parent_id == {}:
                item_parent_id = None

            if item_parent_id == parent_id:
                # Add the current item's name with explicit \n and \t
                result += "\\t" * indent + f"{item['Name']}\\n"
                # Recursively add children
                result += build_hierarchy_string([metadata], parent_id=item["ID"], indent=indent + 1)
    return result

class FixedListReply(BaseModel):
    dimensions: list
    items: List[dict]

def create_fixed_list(state: ListSubchartState):
    """
    Create a fixed list using the LLM (SpecDec) and filtered metadata.
    """
    current_list = state["List"]
    all_metadata = state["ReportMetadata"]
    chosen_dims = state.get("dimensions", [])

    filtered_metadata = []
    dims = []
    for dim in all_metadata:
        if dim["name"] in chosen_dims:
            new_dim = {
                "name": dim["name"],
                "alias": dim["alias"],
                "dimensionContent": dim.get("dimensionContent", [])
            }
            filtered_metadata.append(new_dim)
            dims.append(dim["name"])

    fixedlist_prompt = load_xml_instructions("fixedlist_prompt.xml")
    system_msg = SystemMessage(content=fixedlist_prompt)

    hierarchystring = build_hierarchy_string(filtered_metadata)
    user_input = {
        "listObject": current_list,
        "dimensions": dims,
        "hierarchy": hierarchystring
    }
    user_msg = HumanMessage(content=json.dumps(user_input, indent=2))

    structured_llm = modelSpec.with_structured_output(
        FixedListReply,
        method="json_mode",
        include_raw=True
    )
    conversation = [system_msg, user_msg]

    # Use safe_invoke_specdec
    output = safe_invoke_specdec(structured_llm, conversation, stream=False)
    parsed_output = output["parsed"]
    final_list = parsed_output.model_dump()

    list_name = current_list.get("list", "Unnamed List")
    named_list = {list_name: final_list}

    return {"JsonLists": [named_list] }

class DynamicListReply(BaseModel):
    dimensions: list
    type: str
    dynamicconfig: dict

def create_dynamic_list(state: ListSubchartState):
    """
    Create a dynamic list using the Versatile model (no token limit logic needed).
    """
    current_list = state["List"]
    all_metadata = state["ReportMetadata"]
    chosen_dims = state.get("dimensions", [])

    filtered_metadata = []
    for dim in all_metadata:
        if dim["name"] in chosen_dims:
            new_dim = {
                "name": dim["name"],
                "alias": dim["alias"],
                "dimensionContent": dim.get("dimensionContent", [])
            }
            filtered_metadata.append(new_dim)

    dynamiclist_prompt = load_xml_instructions("dynamiclist_prompt.xml")
    system_msg = SystemMessage(content=dynamiclist_prompt)

    user_input = {
        "listObject": current_list,
        "filteredMetadata": build_hierarchy_string(filtered_metadata)
    }
    user_msg = HumanMessage(content=json.dumps(user_input, indent=2))

    # This is a Versatile model call, so no special token limit logic
    structured_llm = modelSpec.with_structured_output(
        DynamicListReply,
        method="json_mode",
        include_raw=True
    )
    conversation = [system_msg, user_msg]
    output = safe_invoke_specdec(structured_llm, conversation, stream=False)

    parsed_output = output["parsed"]
    final_list = parsed_output.model_dump()

    list_name = current_list.get("list", "Unnamed List")
    named_list = {list_name: final_list}

    return {"JsonLists": [named_list] }

#########################################################
# 4. Build the subgraph: define nodes & edges
#########################################################
subgraph = StateGraph(ListSubchartState)

# Add nodes
subgraph.add_node("check_if_list_exists", check_if_list_exists)
subgraph.add_node("return_existing_list", return_existing_list)
subgraph.add_node("check_dynamic_or_fixed", check_dynamic_or_fixed)
subgraph.add_node("create_fixed_list", create_fixed_list)
subgraph.add_node("create_dynamic_list", create_dynamic_list)

# Edges
subgraph.add_edge(START, "check_if_list_exists")

def list_exists_routing(state: ListSubchartState):
    if state.get("listExists") is True:
        return "return_existing_list"
    else:
        return "check_dynamic_or_fixed"

subgraph.add_conditional_edges("check_if_list_exists", list_exists_routing,
                               ["return_existing_list", "check_dynamic_or_fixed"])
subgraph.add_edge("return_existing_list", END)

def dynamic_or_fixed_routing(state: ListSubchartState):
    if state.get("listType") == "Dynamic":
        return "create_dynamic_list"
    else:
        return "create_fixed_list"

subgraph.add_conditional_edges("check_dynamic_or_fixed", dynamic_or_fixed_routing,
                               ["create_dynamic_list", "create_fixed_list"])
subgraph.add_edge("create_dynamic_list", END)
subgraph.add_edge("create_fixed_list", END)

# Compile the subchart
generate_list_subchart = subgraph.compile()

def remove_none_members(data):
    """
    Recursively removes all None members from a JSON-like dictionary or list structure.
    """
    if isinstance(data, dict):
        return {k: remove_none_members(v) for k, v in data.items() if v is not None}
    elif isinstance(data, list):
        return [remove_none_members(item) for item in data if item is not None]
    else:
        return data

def consolidate_lists_to_layout(state: OverallState):
    """
    Consolidates all the lists generated by the subgraph into the final layout JSON.
    """
    if "JsonLayout" not in state or state["JsonLayout"] is None:
        state["JsonLayout"] = {}

    if "Lists" not in state["JsonLayout"]:
        state["JsonLayout"]["lists"] = {}

    generated_lists = state.get("JsonLists", {})

    for generated_list in generated_lists:
        if isinstance(generated_list, dict):
            state["JsonLayout"]["lists"].update(generated_list)

    return {"JsonLayout": state["JsonLayout"]}

# Main graph
graph = StateGraph(OverallState)

graph.add_node("generate_layout", generate_layout)
graph.add_node("generate_component", generate_component)
graph.add_node("update_json_layout", update_json_layout)
graph.add_node("gatheruniquelists", gatheruniquelists)
graph.add_node("generate_list_subchart", generate_list_subchart)  # Subchart
graph.add_node("consolidate_lists_to_layout", consolidate_lists_to_layout)

graph.add_edge(START, "generate_layout")
graph.add_conditional_edges("generate_layout", continue_to_components, ["generate_component"])
graph.add_edge("generate_component", "update_json_layout")
graph.add_edge("update_json_layout", "gatheruniquelists")
graph.add_conditional_edges("gatheruniquelists", continue_to_lists, ["generate_list_subchart"])
graph.add_edge("generate_list_subchart", "consolidate_lists_to_layout")
graph.add_edge("consolidate_lists_to_layout", END)

# Compile the main graph
app = graph.compile()


In [8]:
state = {
  "List": {
    "listReference": "categoryList",
    "list": "12periods",
    "createDescription": "A list with 12 rolling months starting from the current period going 11 months back to be used on the x-axis of the chart"
  },
  "listExists": False,
  "listType": "Dynamic",
  "dimensions": [
    "Period"
  ],
  "ReportMetadata": [
    {
      "name": "Category",
      "alias": "Category",
      "order": 3,
      "default": "Actual",
      "visible": False,
      "dimensionContent": []
    },
    {
      "name": "Year",
      "alias": "Year",
      "order": 1,
      "default": 2024,
      "visible": True,
      "dimensionContent": []
    },
    {
      "name": "Period",
      "alias": "Period",
      "order": 2,
      "default": "Oct",
      "visible": True,
      "dimensionContent": [
        {
          "ID": 1,
          "Name": "Jan",
          "Order": 1,
          "ParentID": 13
        },
        {
          "ID": 2,
          "Name": "Feb",
          "Order": 2,
          "ParentID": 13
        },
        {
          "ID": 3,
          "Name": "Mar",
          "Order": 3,
          "ParentID": 13
        },
        {
          "ID": 4,
          "Name": "Apr",
          "Order": 4,
          "ParentID": 14
        },
        {
          "ID": 5,
          "Name": "May",
          "Order": 5,
          "ParentID": 14
        },
        {
          "ID": 6,
          "Name": "Jun",
          "Order": 6,
          "ParentID": 14
        },
        {
          "ID": 7,
          "Name": "Jul",
          "Order": 7,
          "ParentID": 15
        },
        {
          "ID": 8,
          "Name": "Aug",
          "Order": 8,
          "ParentID": 15
        },
        {
          "ID": 9,
          "Name": "Sep",
          "Order": 9,
          "ParentID": 15
        },
        {
          "ID": 10,
          "Name": "Oct",
          "Order": 10,
          "ParentID": 16
        },
        {
          "ID": 11,
          "Name": "Nov",
          "Order": 11,
          "ParentID": 16
        },
        {
          "ID": 12,
          "Name": "Dec",
          "Order": 12,
          "ParentID": 16
        },
        {
          "ID": 13,
          "Name": "Q1",
          "Order": 3,
          "ParentID": 17
        },
        {
          "ID": 14,
          "Name": "Q2",
          "Order": 6,
          "ParentID": 17
        },
        {
          "ID": 15,
          "Name": "Q3",
          "Order": 9,
          "ParentID": 17
        },
        {
          "ID": 16,
          "Name": "Q4",
          "Order": 12,
          "ParentID": 17
        },
        {
          "ID": 17,
          "Name": "Full Year",
          "Order": 12,
          "ParentID": {}
        }
      ]
    },
    {
      "name": "View",
      "alias": "View",
      "order": 2,
      "default": "Periodic",
      "visible": True,
      "dimensionContent": []
    },
    {
      "name": "Account",
      "alias": "Account",
      "order": 6,
      "default": "Gross Sales",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 1059,
          "Name": "Depreciation IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1060,
          "Name": "Interest on term loan",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1061,
          "Name": "Foreign exchange gain/loss",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1063,
          "Name": "Others",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1064,
          "Name": "Depreciation",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1065,
          "Name": "General Office Supplies",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1066,
          "Name": "Advertising and marketing",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1068,
          "Name": "Insurance",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1069,
          "Name": "Internet and Phone",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1070,
          "Name": "Service Charges",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1073,
          "Name": "Rent - IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1074,
          "Name": "Interest on term loan - IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1076,
          "Name": "Legal and professional services",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1077,
          "Name": "Transportation expenses",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1079,
          "Name": "Staff costs salaries and wages",
          "ParentID": 1327,
          "Valuation": "positive"
        },
        {
          "ID": 1081,
          "Name": "Electricity and Cooling",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1082,
          "Name": "Rent",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1084,
          "Name": "Extraordinary Expense",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1087,
          "Name": "Staff costs others",
          "ParentID": 1327,
          "Valuation": "positive"
        },
        {
          "ID": 1090,
          "Name": "Bank charges",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1171,
          "Name": "Deprecation - ROU Asset",
          "ParentID": 1059,
          "Valuation": "positive"
        },
        {
          "ID": 1172,
          "Name": "Interest on Term Deposit",
          "ParentID": 1060,
          "Valuation": "positive"
        },
        {
          "ID": 1173,
          "Name": "Realized Foreign exchange gains and losses",
          "ParentID": 1061,
          "Valuation": "positive"
        },
        {
          "ID": 1198,
          "Name": "Amortization, Software Cost",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1199,
          "Name": "Amortization, Trade Mark/Logo",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1200,
          "Name": "Depreciation - Leasehold Improvement",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1201,
          "Name": "Depreciation, Equipment",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1202,
          "Name": "Depreciation, Furniture & Fixture",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1203,
          "Name": "Depreciation, Vehicles",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1207,
          "Name": "Marketing, Promotions & E-Commerce",
          "ParentID": 1066,
          "Valuation": "positive"
        },
        {
          "ID": 1208,
          "Name": "Shop Use - Marketing/FAHM",
          "ParentID": 1066,
          "Valuation": "positive"
        },
        {
          "ID": 1248,
          "Name": "Rent -IFRS 16 REversal",
          "ParentID": 1073,
          "Valuation": "positive"
        },
        {
          "ID": 1249,
          "Name": "Finance cost - IFRS 16",
          "ParentID": 1074,
          "Valuation": "positive"
        },
        {
          "ID": 1274,
          "Name": "Dividend Received",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1275,
          "Name": "Extraordinary Expenses",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1276,
          "Name": "Extraordinary Income",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1277,
          "Name": "Old Entries",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1278,
          "Name": "Profits on Sales of Fixed Assets",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1325,
          "Name": "Contribution Margin",
          "ParentID": 1326,
          "Valuation": "positive"
        },
        {
          "ID": 1326,
          "Name": "Gross Margin",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1327,
          "Name": "Staff Expenses",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1329,
          "Name": "Rent & Utilities",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1330,
          "Name": "Other Fixed Exp.",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1331,
          "Name": "EBITDA",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1332,
          "Name": "Net Profit",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1333,
          "Name": "Other COGS Total",
          "ParentID": 1326,
          "Valuation": "positive"
        },
        {
          "ID": 1335,
          "Name": "EBITDA Margin",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1336,
          "Name": "Nerw Accounts",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1337,
          "Name": "Bad Debt Expenses",
          "ParentID": 1336,
          "Valuation": "positive"
        },
        {
          "ID": 1338,
          "Name": "Listing Fees",
          "ParentID": 1336,
          "Valuation": "positive"
        }
      ]
    },
    {
      "name": "AnalyticalDim1",
      "alias": "Product",
      "order": 7,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 64,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 65,
          "Name": "Total Cost Center",
          "ParentID": 64
        },
        {
          "ID": 66,
          "Name": "Total Profit Center",
          "ParentID": 64
        },
        {
          "ID": 69,
          "Name": "Project & Maintenance",
          "ParentID": 65
        },
        {
          "ID": 70,
          "Name": "Store-Sheikh Zayed",
          "ParentID": 65
        },
        {
          "ID": 71,
          "Name": "Product Development",
          "ParentID": 65
        },
        {
          "ID": 72,
          "Name": "510150",
          "ParentID": 65
        },
        {
          "ID": 73,
          "Name": "Information Technology",
          "ParentID": 65
        },
        {
          "ID": 74,
          "Name": "CCRXX1",
          "ParentID": 65
        },
        {
          "ID": 75,
          "Name": "Store Dubai Internet City",
          "ParentID": 65
        },
        {
          "ID": 76,
          "Name": "Picker - Logistics",
          "ParentID": 65
        },
        {
          "ID": 77,
          "Name": "Creek Harbour  store pre opex",
          "ParentID": 65
        },
        {
          "ID": 78,
          "Name": "Al Reem Mall Store",
          "ParentID": 65
        },
        {
          "ID": 79,
          "Name": "Emaratization",
          "ParentID": 65
        },
        {
          "ID": 80,
          "Name": "Store-Dubai Silicon Oasis",
          "ParentID": 65
        },
        {
          "ID": 81,
          "Name": "Careem",
          "ParentID": 65
        },
        {
          "ID": 82,
          "Name": "Now Now",
          "ParentID": 65
        },
        {
          "ID": 83,
          "Name": "510321",
          "ParentID": 65
        },
        {
          "ID": 84,
          "Name": "E Commerce",
          "ParentID": 65
        },
        {
          "ID": 85,
          "Name": "510260",
          "ParentID": 65
        },
        {
          "ID": 86,
          "Name": "Petsville-DIP warehouse",
          "ParentID": 65
        },
        {
          "ID": 87,
          "Name": "Umm Suqeim-Iridium Building",
          "ParentID": 65
        },
        {
          "ID": 88,
          "Name": "SHJ",
          "ParentID": 65
        },
        {
          "ID": 89,
          "Name": "510190",
          "ParentID": 65
        },
        {
          "ID": 90,
          "Name": "Marketing",
          "ParentID": 65
        },
        {
          "ID": 91,
          "Name": "JVC",
          "ParentID": 65
        },
        {
          "ID": 92,
          "Name": "510030",
          "ParentID": 65
        },
        {
          "ID": 93,
          "Name": "B2B",
          "ParentID": 65
        },
        {
          "ID": 94,
          "Name": "510331",
          "ParentID": 65
        },
        {
          "ID": 95,
          "Name": "510210",
          "ParentID": 65
        },
        {
          "ID": 97,
          "Name": "510170",
          "ParentID": 65
        },
        {
          "ID": 98,
          "Name": "Mobile Grooming",
          "ParentID": 65
        },
        {
          "ID": 99,
          "Name": "Petsville",
          "ParentID": 65
        },
        {
          "ID": 100,
          "Name": "510132",
          "ParentID": 65
        },
        {
          "ID": 101,
          "Name": "510020",
          "ParentID": 65
        },
        {
          "ID": 102,
          "Name": "510350",
          "ParentID": 65
        },
        {
          "ID": 103,
          "Name": "SIL",
          "ParentID": 65
        },
        {
          "ID": 104,
          "Name": "510050",
          "ParentID": 65
        },
        {
          "ID": 105,
          "Name": "Manager/Supervisor - Logistics",
          "ParentID": 65
        },
        {
          "ID": 106,
          "Name": "New Assets Depr.",
          "ParentID": 65
        },
        {
          "ID": 107,
          "Name": "510130",
          "ParentID": 65
        },
        {
          "ID": 108,
          "Name": "Pre- Opex DIC warehouse",
          "ParentID": 65
        },
        {
          "ID": 109,
          "Name": "510320",
          "ParentID": 65
        },
        {
          "ID": 110,
          "Name": "PPLUS",
          "ParentID": 65
        },
        {
          "ID": 111,
          "Name": "Lucky Vet",
          "ParentID": 65
        },
        {
          "ID": 112,
          "Name": "510330",
          "ParentID": 65
        },
        {
          "ID": 113,
          "Name": "Store-Al Forzan Mall",
          "ParentID": 65
        },
        {
          "ID": 114,
          "Name": "Delivery - Logistics",
          "ParentID": 65
        },
        {
          "ID": 115,
          "Name": "TPS Café",
          "ParentID": 65
        },
        {
          "ID": 116,
          "Name": "Marina Walk Store-Pre opex.",
          "ParentID": 65
        },
        {
          "ID": 117,
          "Name": "DOH1",
          "ParentID": 65
        },
        {
          "ID": 118,
          "Name": "510310",
          "ParentID": 65
        },
        {
          "ID": 119,
          "Name": "Amazone",
          "ParentID": 65
        },
        {
          "ID": 120,
          "Name": "Pre-Opex Umm Suqeim-Iridium Building",
          "ParentID": 65
        },
        {
          "ID": 121,
          "Name": "Pet Relocation",
          "ParentID": 65
        },
        {
          "ID": 122,
          "Name": "Shop Grooming",
          "ParentID": 65
        },
        {
          "ID": 123,
          "Name": "510010",
          "ParentID": 65
        },
        {
          "ID": 124,
          "Name": "Aquatics",
          "ParentID": 65
        },
        {
          "ID": 125,
          "Name": "Warehouse",
          "ParentID": 65
        },
        {
          "ID": 126,
          "Name": "510060",
          "ParentID": 65
        },
        {
          "ID": 127,
          "Name": "Al Reem Mall Store - Pre-opex",
          "ParentID": 65
        },
        {
          "ID": 128,
          "Name": "Qurantine",
          "ParentID": 65
        },
        {
          "ID": 129,
          "Name": "510126",
          "ParentID": 65
        },
        {
          "ID": 130,
          "Name": "CCRXX2",
          "ParentID": 65
        },
        {
          "ID": 131,
          "Name": "510220",
          "ParentID": 65
        },
        {
          "ID": 132,
          "Name": "Packer - Logistics",
          "ParentID": 65
        },
        {
          "ID": 133,
          "Name": "Customer Service",
          "ParentID": 65
        },
        {
          "ID": 134,
          "Name": "Logistics",
          "ParentID": 65
        },
        {
          "ID": 135,
          "Name": "510110",
          "ParentID": 65
        },
        {
          "ID": 136,
          "Name": "Store-Dubai Investment Park",
          "ParentID": 65
        },
        {
          "ID": 137,
          "Name": "510333",
          "ParentID": 65
        },
        {
          "ID": 138,
          "Name": "Marina Walk Store",
          "ParentID": 65
        },
        {
          "ID": 139,
          "Name": "510133",
          "ParentID": 65
        },
        {
          "ID": 140,
          "Name": "x",
          "ParentID": 65
        },
        {
          "ID": 141,
          "Name": "510230",
          "ParentID": 65
        },
        {
          "ID": 142,
          "Name": "510131",
          "ParentID": 65
        },
        {
          "ID": 143,
          "Name": "Talabat",
          "ParentID": 65
        },
        {
          "ID": 144,
          "Name": "Store-Dubai Silicon Oasis pre-opex",
          "ParentID": 65
        },
        {
          "ID": 146,
          "Name": "Retail Store",
          "ParentID": 65
        },
        {
          "ID": 148,
          "Name": "510240",
          "ParentID": 65
        },
        {
          "ID": 149,
          "Name": "FAHRM",
          "ParentID": 65
        },
        {
          "ID": 150,
          "Name": "General/Common Costs",
          "ParentID": 65
        },
        {
          "ID": 151,
          "Name": "EL Grocer",
          "ParentID": 65
        },
        {
          "ID": 152,
          "Name": "Services",
          "ParentID": 65
        },
        {
          "ID": 153,
          "Name": "Instashop",
          "ParentID": 65
        },
        {
          "ID": 154,
          "Name": "Creek harbour Store ",
          "ParentID": 65
        },
        {
          "ID": 155,
          "Name": "Purchasing",
          "ParentID": 65
        },
        {
          "ID": 156,
          "Name": "Planner - Logistics",
          "ParentID": 65
        },
        {
          "ID": 157,
          "Name": "Store-Al Badda Store",
          "ParentID": 65
        },
        {
          "ID": 158,
          "Name": "510135",
          "ParentID": 65
        },
        {
          "ID": 159,
          "Name": "TECH",
          "ParentID": 65
        },
        {
          "ID": 160,
          "Name": "510250",
          "ParentID": 65
        },
        {
          "ID": 161,
          "Name": "Deliveroo",
          "ParentID": 65
        },
        {
          "ID": 162,
          "Name": "510134",
          "ParentID": 65
        },
        {
          "ID": 163,
          "Name": "Store-Ras Al Kaimah",
          "ParentID": 65
        },
        {
          "ID": 164,
          "Name": "510300",
          "ParentID": 65
        },
        {
          "ID": 165,
          "Name": "Swan",
          "ParentID": 65
        },
        {
          "ID": 166,
          "Name": "CCRXX3",
          "ParentID": 65
        },
        {
          "ID": 167,
          "Name": "MTR/ALK",
          "ParentID": 65
        },
        {
          "ID": 168,
          "Name": "Store-Dalma Mall",
          "ParentID": 65
        },
        {
          "ID": 169,
          "Name": "112120",
          "ParentID": 66
        },
        {
          "ID": 170,
          "Name": "123010",
          "ParentID": 66
        },
        {
          "ID": 171,
          "Name": "120040",
          "ParentID": 66
        },
        {
          "ID": 172,
          "Name": "112170",
          "ParentID": 66
        },
        {
          "ID": 173,
          "Name": "112190",
          "ParentID": 66
        },
        {
          "ID": 174,
          "Name": "112180",
          "ParentID": 66
        },
        {
          "ID": 175,
          "Name": "123030",
          "ParentID": 66
        },
        {
          "ID": 176,
          "Name": "112160",
          "ParentID": 66
        },
        {
          "ID": 177,
          "Name": "122000",
          "ParentID": 66
        },
        {
          "ID": 178,
          "Name": "120010",
          "ParentID": 66
        },
        {
          "ID": 179,
          "Name": "Petsville",
          "ParentID": 66
        },
        {
          "ID": 180,
          "Name": "110000",
          "ParentID": 66
        },
        {
          "ID": 181,
          "Name": "100000",
          "ParentID": 66
        },
        {
          "ID": 182,
          "Name": "TPS Café",
          "ParentID": 66
        },
        {
          "ID": 183,
          "Name": "112185",
          "ParentID": 66
        },
        {
          "ID": 184,
          "Name": "112125",
          "ParentID": 66
        },
        {
          "ID": 185,
          "Name": "111000",
          "ParentID": 66
        },
        {
          "ID": 186,
          "Name": "112150",
          "ParentID": 66
        },
        {
          "ID": 187,
          "Name": "123040",
          "ParentID": 66
        },
        {
          "ID": 188,
          "Name": "123020",
          "ParentID": 66
        },
        {
          "ID": 189,
          "Name": "112130",
          "ParentID": 66
        },
        {
          "ID": 190,
          "Name": "112140",
          "ParentID": 66
        },
        {
          "ID": 191,
          "Name": "510040",
          "ParentID": 65
        },
        {
          "ID": 192,
          "Name": "510070",
          "ParentID": 65
        },
        {
          "ID": 193,
          "Name": "510080",
          "ParentID": 65
        },
        {
          "ID": 194,
          "Name": "510090",
          "ParentID": 65
        },
        {
          "ID": 195,
          "Name": "510115",
          "ParentID": 65
        },
        {
          "ID": 196,
          "Name": "510120",
          "ParentID": 65
        },
        {
          "ID": 197,
          "Name": "510125",
          "ParentID": 65
        },
        {
          "ID": 198,
          "Name": "510140",
          "ParentID": 65
        },
        {
          "ID": 199,
          "Name": "510160",
          "ParentID": 65
        },
        {
          "ID": 200,
          "Name": "510180",
          "ParentID": 65
        },
        {
          "ID": 201,
          "Name": "510200",
          "ParentID": 65
        },
        {
          "ID": 202,
          "Name": "510270",
          "ParentID": 65
        },
        {
          "ID": 203,
          "Name": "510280",
          "ParentID": 65
        },
        {
          "ID": 204,
          "Name": "510290",
          "ParentID": 65
        },
        {
          "ID": 205,
          "Name": "510295",
          "ParentID": 65
        },
        {
          "ID": 206,
          "Name": "510332",
          "ParentID": 65
        },
        {
          "ID": 207,
          "Name": "510334",
          "ParentID": 65
        },
        {
          "ID": 208,
          "Name": "510335",
          "ParentID": 65
        },
        {
          "ID": 209,
          "Name": "510336",
          "ParentID": 65
        },
        {
          "ID": 210,
          "Name": "510337",
          "ParentID": 65
        },
        {
          "ID": 211,
          "Name": "510340",
          "ParentID": 65
        },
        {
          "ID": 212,
          "Name": "510360",
          "ParentID": 65
        },
        {
          "ID": 213,
          "Name": "510361",
          "ParentID": 65
        },
        {
          "ID": 214,
          "Name": "600000",
          "ParentID": 65
        },
        {
          "ID": 215,
          "Name": "No Cost Center",
          "ParentID": 65
        },
        {
          "ID": 216,
          "Name": "No Profit Center",
          "ParentID": 66
        }
      ]
    },
    {
      "name": "AnalyticalDim2",
      "alias": "Investment",
      "order": 8,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 92,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 93,
          "Name": "P&M",
          "ParentID": 128
        },
        {
          "ID": 94,
          "Name": "#N/A",
          "ParentID": 92
        },
        {
          "ID": 95,
          "Name": "Mobile Grooming",
          "ParentID": 128
        },
        {
          "ID": 96,
          "Name": "Direct Online",
          "ParentID": 129
        },
        {
          "ID": 97,
          "Name": "Backoffice",
          "ParentID": 92
        },
        {
          "ID": 98,
          "Name": "Shop Grooming",
          "ParentID": 128
        },
        {
          "ID": 99,
          "Name": "Petsville",
          "ParentID": 92
        },
        {
          "ID": 100,
          "Name": "TPS Café",
          "ParentID": 92
        },
        {
          "ID": 101,
          "Name": "Ind. Online",
          "ParentID": 129
        },
        {
          "ID": 102,
          "Name": "Shop Sales",
          "ParentID": 92
        },
        {
          "ID": 103,
          "Name": "Pet Relocation",
          "ParentID": 128
        },
        {
          "ID": 104,
          "Name": "B2B",
          "ParentID": 128
        },
        {
          "ID": 105,
          "Name": "0",
          "ParentID": 92
        },
        {
          "ID": 108,
          "Name": "Amazon",
          "ParentID": 101
        },
        {
          "ID": 109,
          "Name": "Amazone Retail",
          "ParentID": 101
        },
        {
          "ID": 112,
          "Name": "BlazeApp",
          "ParentID": 101
        },
        {
          "ID": 113,
          "Name": "Careem",
          "ParentID": 101
        },
        {
          "ID": 114,
          "Name": "Deliveroo",
          "ParentID": 101
        },
        {
          "ID": 116,
          "Name": "El Grocer",
          "ParentID": 101
        },
        {
          "ID": 117,
          "Name": "Instashop",
          "ParentID": 101
        },
        {
          "ID": 119,
          "Name": "Now Now",
          "ParentID": 101
        },
        {
          "ID": 125,
          "Name": "Swan",
          "ParentID": 101
        },
        {
          "ID": 126,
          "Name": "Talabat",
          "ParentID": 101
        },
        {
          "ID": 128,
          "Name": "Services and B2B",
          "ParentID": 92
        },
        {
          "ID": 129,
          "Name": "Ecommerce",
          "ParentID": 92
        }
      ]
    },
    {
      "name": "Entity",
      "alias": "Entity",
      "order": 0,
      "default": "The Petshop",
      "visible": True,
      "dimensionContent": [
        {
          "ID": 120,
          "Name": "Petsville",
          "ParentID": 123
        },
        {
          "ID": 121,
          "Name": "TPS",
          "ParentID": 124
        },
        {
          "ID": 122,
          "Name": "TPS Café",
          "ParentID": 124
        },
        {
          "ID": 123,
          "Name": "The Petshop",
          "ParentID": {}
        },
        {
          "ID": 124,
          "Name": "TPS+",
          "ParentID": 123
        },
        {
          "ID": 125,
          "Name": "TPS Qatar",
          "ParentID": 124
        }
      ]
    },
    {
      "name": "AnalyticalDim3",
      "alias": "AnalyticalDim3",
      "order": 9,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 37,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 38,
          "Name": "UMSQ",
          "ParentID": 37
        },
        {
          "ID": 39,
          "Name": "HQW2",
          "ParentID": 37
        },
        {
          "ID": 40,
          "Name": "PRO-MAIN",
          "ParentID": 37
        },
        {
          "ID": 41,
          "Name": "SHJ",
          "ParentID": 37
        },
        {
          "ID": 42,
          "Name": "DOH1",
          "ParentID": 37
        },
        {
          "ID": 43,
          "Name": "AL QUOZ",
          "ParentID": 37
        },
        {
          "ID": 44,
          "Name": "CAFE",
          "ParentID": 37
        },
        {
          "ID": 45,
          "Name": "MRI",
          "ParentID": 37
        },
        {
          "ID": 46,
          "Name": "x",
          "ParentID": 37
        },
        {
          "ID": 47,
          "Name": "DAMAGE",
          "ParentID": 37
        },
        {
          "ID": 48,
          "Name": "DSO",
          "ParentID": 37
        },
        {
          "ID": 49,
          "Name": "JVC",
          "ParentID": 37
        },
        {
          "ID": 50,
          "Name": "DLM",
          "ParentID": 37
        },
        {
          "ID": 51,
          "Name": "FZN",
          "ParentID": 37
        },
        {
          "ID": 52,
          "Name": "REM",
          "ParentID": 37
        },
        {
          "ID": 53,
          "Name": "HQWCD",
          "ParentID": 37
        },
        {
          "ID": 54,
          "Name": "QUAR",
          "ParentID": 37
        },
        {
          "ID": 55,
          "Name": "HO",
          "ParentID": 37
        },
        {
          "ID": 56,
          "Name": "CREEK",
          "ParentID": 37
        },
        {
          "ID": 57,
          "Name": "GRM",
          "ParentID": 37
        },
        {
          "ID": 58,
          "Name": "IMP-SHORT",
          "ParentID": 37
        },
        {
          "ID": 59,
          "Name": "MRK",
          "ParentID": 37
        },
        {
          "ID": 60,
          "Name": "DIC",
          "ParentID": 37
        },
        {
          "ID": 61,
          "Name": "PPLUS",
          "ParentID": 37
        },
        {
          "ID": 62,
          "Name": "HQW",
          "ParentID": 37
        },
        {
          "ID": 63,
          "Name": "SZR",
          "ParentID": 37
        },
        {
          "ID": 64,
          "Name": "MEAISEM",
          "ParentID": 37
        },
        {
          "ID": 65,
          "Name": "PRL",
          "ParentID": 37
        },
        {
          "ID": 66,
          "Name": "RAK",
          "ParentID": 37
        },
        {
          "ID": 67,
          "Name": "VILLA AJ",
          "ParentID": 37
        },
        {
          "ID": 68,
          "Name": "DIP",
          "ParentID": 37
        },
        {
          "ID": 69,
          "Name": "MTR/ALK",
          "ParentID": 37
        },
        {
          "ID": 70,
          "Name": "PALM",
          "ParentID": 37
        },
        {
          "ID": 71,
          "Name": "WSL",
          "ParentID": 37
        },
        {
          "ID": 72,
          "Name": "MOBILE",
          "ParentID": 37
        },
        {
          "ID": 73,
          "Name": "No Location",
          "ParentID": 37
        },
        {
          "ID": 74,
          "Name": "XX2",
          "ParentID": 37
        },
        {
          "ID": 75,
          "Name": "XX1",
          "ParentID": 37
        },
        {
          "ID": 76,
          "Name": "XX3",
          "ParentID": 37
        }
      ]
    },
    {
      "name": "AnalyticalDim4",
      "alias": "Normilization",
      "order": 11,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 40,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 41,
          "Name": "Normilization",
          "ParentID": 40
        },
        {
          "ID": 42,
          "Name": "Pre-OPEX",
          "ParentID": 40
        },
        {
          "ID": 43,
          "Name": "Accounting Treat - Visa Prepaid Write-off ",
          "ParentID": 41
        },
        {
          "ID": 44,
          "Name": "Leave Provision - Accounting Adjustment",
          "ParentID": 41
        },
        {
          "ID": 45,
          "Name": "Legal/Court Case",
          "ParentID": 41
        },
        {
          "ID": 46,
          "Name": "Lucky Vet - Acquisition Expenses",
          "ParentID": 41
        },
        {
          "ID": 47,
          "Name": "Nishe Services for VAT and Corp. Tax",
          "ParentID": 41
        },
        {
          "ID": 48,
          "Name": "Tech Recruitment",
          "ParentID": 41
        },
        {
          "ID": 49,
          "Name": "Pre-OPEX - Petsville DIP",
          "ParentID": 42
        },
        {
          "ID": 50,
          "Name": "Pre-OPEX - Silicon Oasis",
          "ParentID": 42
        },
        {
          "ID": 51,
          "Name": "Pre-OPEX Creek",
          "ParentID": 42
        },
        {
          "ID": 52,
          "Name": "Pre-OPEX Dubai Marina",
          "ParentID": 42
        },
        {
          "ID": 53,
          "Name": "Pre-OPEX Reem",
          "ParentID": 42
        },
        {
          "ID": 54,
          "Name": "Pre-OPEX Reem (Store Opening)",
          "ParentID": 42
        },
        {
          "ID": 55,
          "Name": "Pre-OPEX Umm Suqeim",
          "ParentID": 42
        },
        {
          "ID": 56,
          "Name": "Pre-Opex Warehouse",
          "ParentID": 42
        },
        {
          "ID": 57,
          "Name": "Pre-Opex Warehouse (PV)",
          "ParentID": 42
        },
        {
          "ID": 59,
          "Name": "No Normalization",
          "ParentID": 40
        }
      ]
    },
    {
      "name": "Category",
      "alias": "Category",
      "order": 3,
      "default": "Actual",
      "visible": False,
      "dimensionContent": []
    },
    {
      "name": "Year",
      "alias": "Year",
      "order": 1,
      "default": 2024,
      "visible": True,
      "dimensionContent": []
    },
    {
      "name": "Period",
      "alias": "Period",
      "order": 2,
      "default": "Oct",
      "visible": True,
      "dimensionContent": [
        {
          "ID": 1,
          "Name": "Jan",
          "Order": 1,
          "ParentID": 13
        },
        {
          "ID": 2,
          "Name": "Feb",
          "Order": 2,
          "ParentID": 13
        },
        {
          "ID": 3,
          "Name": "Mar",
          "Order": 3,
          "ParentID": 13
        },
        {
          "ID": 4,
          "Name": "Apr",
          "Order": 4,
          "ParentID": 14
        },
        {
          "ID": 5,
          "Name": "May",
          "Order": 5,
          "ParentID": 14
        },
        {
          "ID": 6,
          "Name": "Jun",
          "Order": 6,
          "ParentID": 14
        },
        {
          "ID": 7,
          "Name": "Jul",
          "Order": 7,
          "ParentID": 15
        },
        {
          "ID": 8,
          "Name": "Aug",
          "Order": 8,
          "ParentID": 15
        },
        {
          "ID": 9,
          "Name": "Sep",
          "Order": 9,
          "ParentID": 15
        },
        {
          "ID": 10,
          "Name": "Oct",
          "Order": 10,
          "ParentID": 16
        },
        {
          "ID": 11,
          "Name": "Nov",
          "Order": 11,
          "ParentID": 16
        },
        {
          "ID": 12,
          "Name": "Dec",
          "Order": 12,
          "ParentID": 16
        },
        {
          "ID": 13,
          "Name": "Q1",
          "Order": 3,
          "ParentID": 17
        },
        {
          "ID": 14,
          "Name": "Q2",
          "Order": 6,
          "ParentID": 17
        },
        {
          "ID": 15,
          "Name": "Q3",
          "Order": 9,
          "ParentID": 17
        },
        {
          "ID": 16,
          "Name": "Q4",
          "Order": 12,
          "ParentID": 17
        },
        {
          "ID": 17,
          "Name": "Full Year",
          "Order": 12,
          "ParentID": {}
        }
      ]
    },
    {
      "name": "View",
      "alias": "View",
      "order": 2,
      "default": "Periodic",
      "visible": True,
      "dimensionContent": []
    },
    {
      "name": "Account",
      "alias": "Account",
      "order": 6,
      "default": "Gross Sales",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 1059,
          "Name": "Depreciation IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1060,
          "Name": "Interest on term loan",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1061,
          "Name": "Foreign exchange gain/loss",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1063,
          "Name": "Others",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1064,
          "Name": "Depreciation",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1065,
          "Name": "General Office Supplies",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1066,
          "Name": "Advertising and marketing",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1068,
          "Name": "Insurance",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1069,
          "Name": "Internet and Phone",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1070,
          "Name": "Service Charges",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1073,
          "Name": "Rent - IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1074,
          "Name": "Interest on term loan - IFRS16",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1076,
          "Name": "Legal and professional services",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1077,
          "Name": "Transportation expenses",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1079,
          "Name": "Staff costs salaries and wages",
          "ParentID": 1327,
          "Valuation": "positive"
        },
        {
          "ID": 1081,
          "Name": "Electricity and Cooling",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1082,
          "Name": "Rent",
          "ParentID": 1329,
          "Valuation": "positive"
        },
        {
          "ID": 1084,
          "Name": "Extraordinary Expense",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1087,
          "Name": "Staff costs others",
          "ParentID": 1327,
          "Valuation": "positive"
        },
        {
          "ID": 1090,
          "Name": "Bank charges",
          "ParentID": 1330,
          "Valuation": "positive"
        },
        {
          "ID": 1171,
          "Name": "Deprecation - ROU Asset",
          "ParentID": 1059,
          "Valuation": "positive"
        },
        {
          "ID": 1172,
          "Name": "Interest on Term Deposit",
          "ParentID": 1060,
          "Valuation": "positive"
        },
        {
          "ID": 1173,
          "Name": "Realized Foreign exchange gains and losses",
          "ParentID": 1061,
          "Valuation": "positive"
        },
        {
          "ID": 1198,
          "Name": "Amortization, Software Cost",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1199,
          "Name": "Amortization, Trade Mark/Logo",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1200,
          "Name": "Depreciation - Leasehold Improvement",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1201,
          "Name": "Depreciation, Equipment",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1202,
          "Name": "Depreciation, Furniture & Fixture",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1203,
          "Name": "Depreciation, Vehicles",
          "ParentID": 1064,
          "Valuation": "positive"
        },
        {
          "ID": 1207,
          "Name": "Marketing, Promotions & E-Commerce",
          "ParentID": 1066,
          "Valuation": "positive"
        },
        {
          "ID": 1208,
          "Name": "Shop Use - Marketing/FAHM",
          "ParentID": 1066,
          "Valuation": "positive"
        },
        {
          "ID": 1248,
          "Name": "Rent -IFRS 16 REversal",
          "ParentID": 1073,
          "Valuation": "positive"
        },
        {
          "ID": 1249,
          "Name": "Finance cost - IFRS 16",
          "ParentID": 1074,
          "Valuation": "positive"
        },
        {
          "ID": 1274,
          "Name": "Dividend Received",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1275,
          "Name": "Extraordinary Expenses",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1276,
          "Name": "Extraordinary Income",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1277,
          "Name": "Old Entries",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1278,
          "Name": "Profits on Sales of Fixed Assets",
          "ParentID": 1084,
          "Valuation": "positive"
        },
        {
          "ID": 1325,
          "Name": "Contribution Margin",
          "ParentID": 1326,
          "Valuation": "positive"
        },
        {
          "ID": 1326,
          "Name": "Gross Margin",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1327,
          "Name": "Staff Expenses",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1329,
          "Name": "Rent & Utilities",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1330,
          "Name": "Other Fixed Exp.",
          "ParentID": 1331,
          "Valuation": "positive"
        },
        {
          "ID": 1331,
          "Name": "EBITDA",
          "ParentID": 1332,
          "Valuation": "positive"
        },
        {
          "ID": 1332,
          "Name": "Net Profit",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1333,
          "Name": "Other COGS Total",
          "ParentID": 1326,
          "Valuation": "positive"
        },
        {
          "ID": 1335,
          "Name": "EBITDA Margin",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1336,
          "Name": "Nerw Accounts",
          "ParentID": {},
          "Valuation": "positive"
        },
        {
          "ID": 1337,
          "Name": "Bad Debt Expenses",
          "ParentID": 1336,
          "Valuation": "positive"
        },
        {
          "ID": 1338,
          "Name": "Listing Fees",
          "ParentID": 1336,
          "Valuation": "positive"
        }
      ]
    },
    {
      "name": "AnalyticalDim1",
      "alias": "Product",
      "order": 7,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 64,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 65,
          "Name": "Total Cost Center",
          "ParentID": 64
        },
        {
          "ID": 66,
          "Name": "Total Profit Center",
          "ParentID": 64
        },
        {
          "ID": 69,
          "Name": "Project & Maintenance",
          "ParentID": 65
        },
        {
          "ID": 70,
          "Name": "Store-Sheikh Zayed",
          "ParentID": 65
        },
        {
          "ID": 71,
          "Name": "Product Development",
          "ParentID": 65
        },
        {
          "ID": 72,
          "Name": "510150",
          "ParentID": 65
        },
        {
          "ID": 73,
          "Name": "Information Technology",
          "ParentID": 65
        },
        {
          "ID": 74,
          "Name": "CCRXX1",
          "ParentID": 65
        },
        {
          "ID": 75,
          "Name": "Store Dubai Internet City",
          "ParentID": 65
        },
        {
          "ID": 76,
          "Name": "Picker - Logistics",
          "ParentID": 65
        },
        {
          "ID": 77,
          "Name": "Creek Harbour  store pre opex",
          "ParentID": 65
        },
        {
          "ID": 78,
          "Name": "Al Reem Mall Store",
          "ParentID": 65
        },
        {
          "ID": 79,
          "Name": "Emaratization",
          "ParentID": 65
        },
        {
          "ID": 80,
          "Name": "Store-Dubai Silicon Oasis",
          "ParentID": 65
        },
        {
          "ID": 81,
          "Name": "Careem",
          "ParentID": 65
        },
        {
          "ID": 82,
          "Name": "Now Now",
          "ParentID": 65
        },
        {
          "ID": 83,
          "Name": "510321",
          "ParentID": 65
        },
        {
          "ID": 84,
          "Name": "E Commerce",
          "ParentID": 65
        },
        {
          "ID": 85,
          "Name": "510260",
          "ParentID": 65
        },
        {
          "ID": 86,
          "Name": "Petsville-DIP warehouse",
          "ParentID": 65
        },
        {
          "ID": 87,
          "Name": "Umm Suqeim-Iridium Building",
          "ParentID": 65
        },
        {
          "ID": 88,
          "Name": "SHJ",
          "ParentID": 65
        },
        {
          "ID": 89,
          "Name": "510190",
          "ParentID": 65
        },
        {
          "ID": 90,
          "Name": "Marketing",
          "ParentID": 65
        },
        {
          "ID": 91,
          "Name": "JVC",
          "ParentID": 65
        },
        {
          "ID": 92,
          "Name": "510030",
          "ParentID": 65
        },
        {
          "ID": 93,
          "Name": "B2B",
          "ParentID": 65
        },
        {
          "ID": 94,
          "Name": "510331",
          "ParentID": 65
        },
        {
          "ID": 95,
          "Name": "510210",
          "ParentID": 65
        },
        {
          "ID": 97,
          "Name": "510170",
          "ParentID": 65
        },
        {
          "ID": 98,
          "Name": "Mobile Grooming",
          "ParentID": 65
        },
        {
          "ID": 99,
          "Name": "Petsville",
          "ParentID": 65
        },
        {
          "ID": 100,
          "Name": "510132",
          "ParentID": 65
        },
        {
          "ID": 101,
          "Name": "510020",
          "ParentID": 65
        },
        {
          "ID": 102,
          "Name": "510350",
          "ParentID": 65
        },
        {
          "ID": 103,
          "Name": "SIL",
          "ParentID": 65
        },
        {
          "ID": 104,
          "Name": "510050",
          "ParentID": 65
        },
        {
          "ID": 105,
          "Name": "Manager/Supervisor - Logistics",
          "ParentID": 65
        },
        {
          "ID": 106,
          "Name": "New Assets Depr.",
          "ParentID": 65
        },
        {
          "ID": 107,
          "Name": "510130",
          "ParentID": 65
        },
        {
          "ID": 108,
          "Name": "Pre- Opex DIC warehouse",
          "ParentID": 65
        },
        {
          "ID": 109,
          "Name": "510320",
          "ParentID": 65
        },
        {
          "ID": 110,
          "Name": "PPLUS",
          "ParentID": 65
        },
        {
          "ID": 111,
          "Name": "Lucky Vet",
          "ParentID": 65
        },
        {
          "ID": 112,
          "Name": "510330",
          "ParentID": 65
        },
        {
          "ID": 113,
          "Name": "Store-Al Forzan Mall",
          "ParentID": 65
        },
        {
          "ID": 114,
          "Name": "Delivery - Logistics",
          "ParentID": 65
        },
        {
          "ID": 115,
          "Name": "TPS Café",
          "ParentID": 65
        },
        {
          "ID": 116,
          "Name": "Marina Walk Store-Pre opex.",
          "ParentID": 65
        },
        {
          "ID": 117,
          "Name": "DOH1",
          "ParentID": 65
        },
        {
          "ID": 118,
          "Name": "510310",
          "ParentID": 65
        },
        {
          "ID": 119,
          "Name": "Amazone",
          "ParentID": 65
        },
        {
          "ID": 120,
          "Name": "Pre-Opex Umm Suqeim-Iridium Building",
          "ParentID": 65
        },
        {
          "ID": 121,
          "Name": "Pet Relocation",
          "ParentID": 65
        },
        {
          "ID": 122,
          "Name": "Shop Grooming",
          "ParentID": 65
        },
        {
          "ID": 123,
          "Name": "510010",
          "ParentID": 65
        },
        {
          "ID": 124,
          "Name": "Aquatics",
          "ParentID": 65
        },
        {
          "ID": 125,
          "Name": "Warehouse",
          "ParentID": 65
        },
        {
          "ID": 126,
          "Name": "510060",
          "ParentID": 65
        },
        {
          "ID": 127,
          "Name": "Al Reem Mall Store - Pre-opex",
          "ParentID": 65
        },
        {
          "ID": 128,
          "Name": "Qurantine",
          "ParentID": 65
        },
        {
          "ID": 129,
          "Name": "510126",
          "ParentID": 65
        },
        {
          "ID": 130,
          "Name": "CCRXX2",
          "ParentID": 65
        },
        {
          "ID": 131,
          "Name": "510220",
          "ParentID": 65
        },
        {
          "ID": 132,
          "Name": "Packer - Logistics",
          "ParentID": 65
        },
        {
          "ID": 133,
          "Name": "Customer Service",
          "ParentID": 65
        },
        {
          "ID": 134,
          "Name": "Logistics",
          "ParentID": 65
        },
        {
          "ID": 135,
          "Name": "510110",
          "ParentID": 65
        },
        {
          "ID": 136,
          "Name": "Store-Dubai Investment Park",
          "ParentID": 65
        },
        {
          "ID": 137,
          "Name": "510333",
          "ParentID": 65
        },
        {
          "ID": 138,
          "Name": "Marina Walk Store",
          "ParentID": 65
        },
        {
          "ID": 139,
          "Name": "510133",
          "ParentID": 65
        },
        {
          "ID": 140,
          "Name": "x",
          "ParentID": 65
        },
        {
          "ID": 141,
          "Name": "510230",
          "ParentID": 65
        },
        {
          "ID": 142,
          "Name": "510131",
          "ParentID": 65
        },
        {
          "ID": 143,
          "Name": "Talabat",
          "ParentID": 65
        },
        {
          "ID": 144,
          "Name": "Store-Dubai Silicon Oasis pre-opex",
          "ParentID": 65
        },
        {
          "ID": 146,
          "Name": "Retail Store",
          "ParentID": 65
        },
        {
          "ID": 148,
          "Name": "510240",
          "ParentID": 65
        },
        {
          "ID": 149,
          "Name": "FAHRM",
          "ParentID": 65
        },
        {
          "ID": 150,
          "Name": "General/Common Costs",
          "ParentID": 65
        },
        {
          "ID": 151,
          "Name": "EL Grocer",
          "ParentID": 65
        },
        {
          "ID": 152,
          "Name": "Services",
          "ParentID": 65
        },
        {
          "ID": 153,
          "Name": "Instashop",
          "ParentID": 65
        },
        {
          "ID": 154,
          "Name": "Creek harbour Store ",
          "ParentID": 65
        },
        {
          "ID": 155,
          "Name": "Purchasing",
          "ParentID": 65
        },
        {
          "ID": 156,
          "Name": "Planner - Logistics",
          "ParentID": 65
        },
        {
          "ID": 157,
          "Name": "Store-Al Badda Store",
          "ParentID": 65
        },
        {
          "ID": 158,
          "Name": "510135",
          "ParentID": 65
        },
        {
          "ID": 159,
          "Name": "TECH",
          "ParentID": 65
        },
        {
          "ID": 160,
          "Name": "510250",
          "ParentID": 65
        },
        {
          "ID": 161,
          "Name": "Deliveroo",
          "ParentID": 65
        },
        {
          "ID": 162,
          "Name": "510134",
          "ParentID": 65
        },
        {
          "ID": 163,
          "Name": "Store-Ras Al Kaimah",
          "ParentID": 65
        },
        {
          "ID": 164,
          "Name": "510300",
          "ParentID": 65
        },
        {
          "ID": 165,
          "Name": "Swan",
          "ParentID": 65
        },
        {
          "ID": 166,
          "Name": "CCRXX3",
          "ParentID": 65
        },
        {
          "ID": 167,
          "Name": "MTR/ALK",
          "ParentID": 65
        },
        {
          "ID": 168,
          "Name": "Store-Dalma Mall",
          "ParentID": 65
        },
        {
          "ID": 169,
          "Name": "112120",
          "ParentID": 66
        },
        {
          "ID": 170,
          "Name": "123010",
          "ParentID": 66
        },
        {
          "ID": 171,
          "Name": "120040",
          "ParentID": 66
        },
        {
          "ID": 172,
          "Name": "112170",
          "ParentID": 66
        },
        {
          "ID": 173,
          "Name": "112190",
          "ParentID": 66
        },
        {
          "ID": 174,
          "Name": "112180",
          "ParentID": 66
        },
        {
          "ID": 175,
          "Name": "123030",
          "ParentID": 66
        },
        {
          "ID": 176,
          "Name": "112160",
          "ParentID": 66
        },
        {
          "ID": 177,
          "Name": "122000",
          "ParentID": 66
        },
        {
          "ID": 178,
          "Name": "120010",
          "ParentID": 66
        },
        {
          "ID": 179,
          "Name": "Petsville",
          "ParentID": 66
        },
        {
          "ID": 180,
          "Name": "110000",
          "ParentID": 66
        },
        {
          "ID": 181,
          "Name": "100000",
          "ParentID": 66
        },
        {
          "ID": 182,
          "Name": "TPS Café",
          "ParentID": 66
        },
        {
          "ID": 183,
          "Name": "112185",
          "ParentID": 66
        },
        {
          "ID": 184,
          "Name": "112125",
          "ParentID": 66
        },
        {
          "ID": 185,
          "Name": "111000",
          "ParentID": 66
        },
        {
          "ID": 186,
          "Name": "112150",
          "ParentID": 66
        },
        {
          "ID": 187,
          "Name": "123040",
          "ParentID": 66
        },
        {
          "ID": 188,
          "Name": "123020",
          "ParentID": 66
        },
        {
          "ID": 189,
          "Name": "112130",
          "ParentID": 66
        },
        {
          "ID": 190,
          "Name": "112140",
          "ParentID": 66
        },
        {
          "ID": 191,
          "Name": "510040",
          "ParentID": 65
        },
        {
          "ID": 192,
          "Name": "510070",
          "ParentID": 65
        },
        {
          "ID": 193,
          "Name": "510080",
          "ParentID": 65
        },
        {
          "ID": 194,
          "Name": "510090",
          "ParentID": 65
        },
        {
          "ID": 195,
          "Name": "510115",
          "ParentID": 65
        },
        {
          "ID": 196,
          "Name": "510120",
          "ParentID": 65
        },
        {
          "ID": 197,
          "Name": "510125",
          "ParentID": 65
        },
        {
          "ID": 198,
          "Name": "510140",
          "ParentID": 65
        },
        {
          "ID": 199,
          "Name": "510160",
          "ParentID": 65
        },
        {
          "ID": 200,
          "Name": "510180",
          "ParentID": 65
        },
        {
          "ID": 201,
          "Name": "510200",
          "ParentID": 65
        },
        {
          "ID": 202,
          "Name": "510270",
          "ParentID": 65
        },
        {
          "ID": 203,
          "Name": "510280",
          "ParentID": 65
        },
        {
          "ID": 204,
          "Name": "510290",
          "ParentID": 65
        },
        {
          "ID": 205,
          "Name": "510295",
          "ParentID": 65
        },
        {
          "ID": 206,
          "Name": "510332",
          "ParentID": 65
        },
        {
          "ID": 207,
          "Name": "510334",
          "ParentID": 65
        },
        {
          "ID": 208,
          "Name": "510335",
          "ParentID": 65
        },
        {
          "ID": 209,
          "Name": "510336",
          "ParentID": 65
        },
        {
          "ID": 210,
          "Name": "510337",
          "ParentID": 65
        },
        {
          "ID": 211,
          "Name": "510340",
          "ParentID": 65
        },
        {
          "ID": 212,
          "Name": "510360",
          "ParentID": 65
        },
        {
          "ID": 213,
          "Name": "510361",
          "ParentID": 65
        },
        {
          "ID": 214,
          "Name": "600000",
          "ParentID": 65
        },
        {
          "ID": 215,
          "Name": "No Cost Center",
          "ParentID": 65
        },
        {
          "ID": 216,
          "Name": "No Profit Center",
          "ParentID": 66
        }
      ]
    },
    {
      "name": "AnalyticalDim2",
      "alias": "Investment",
      "order": 8,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 92,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 93,
          "Name": "P&M",
          "ParentID": 128
        },
        {
          "ID": 94,
          "Name": "#N/A",
          "ParentID": 92
        },
        {
          "ID": 95,
          "Name": "Mobile Grooming",
          "ParentID": 128
        },
        {
          "ID": 96,
          "Name": "Direct Online",
          "ParentID": 129
        },
        {
          "ID": 97,
          "Name": "Backoffice",
          "ParentID": 92
        },
        {
          "ID": 98,
          "Name": "Shop Grooming",
          "ParentID": 128
        },
        {
          "ID": 99,
          "Name": "Petsville",
          "ParentID": 92
        },
        {
          "ID": 100,
          "Name": "TPS Café",
          "ParentID": 92
        },
        {
          "ID": 101,
          "Name": "Ind. Online",
          "ParentID": 129
        },
        {
          "ID": 102,
          "Name": "Shop Sales",
          "ParentID": 92
        },
        {
          "ID": 103,
          "Name": "Pet Relocation",
          "ParentID": 128
        },
        {
          "ID": 104,
          "Name": "B2B",
          "ParentID": 128
        },
        {
          "ID": 105,
          "Name": "0",
          "ParentID": 92
        },
        {
          "ID": 108,
          "Name": "Amazon",
          "ParentID": 101
        },
        {
          "ID": 109,
          "Name": "Amazone Retail",
          "ParentID": 101
        },
        {
          "ID": 112,
          "Name": "BlazeApp",
          "ParentID": 101
        },
        {
          "ID": 113,
          "Name": "Careem",
          "ParentID": 101
        },
        {
          "ID": 114,
          "Name": "Deliveroo",
          "ParentID": 101
        },
        {
          "ID": 116,
          "Name": "El Grocer",
          "ParentID": 101
        },
        {
          "ID": 117,
          "Name": "Instashop",
          "ParentID": 101
        },
        {
          "ID": 119,
          "Name": "Now Now",
          "ParentID": 101
        },
        {
          "ID": 125,
          "Name": "Swan",
          "ParentID": 101
        },
        {
          "ID": 126,
          "Name": "Talabat",
          "ParentID": 101
        },
        {
          "ID": 128,
          "Name": "Services and B2B",
          "ParentID": 92
        },
        {
          "ID": 129,
          "Name": "Ecommerce",
          "ParentID": 92
        }
      ]
    },
    {
      "name": "Entity",
      "alias": "Entity",
      "order": 0,
      "default": "The Petshop",
      "visible": True,
      "dimensionContent": [
        {
          "ID": 120,
          "Name": "Petsville",
          "ParentID": 123
        },
        {
          "ID": 121,
          "Name": "TPS",
          "ParentID": 124
        },
        {
          "ID": 122,
          "Name": "TPS Café",
          "ParentID": 124
        },
        {
          "ID": 123,
          "Name": "The Petshop",
          "ParentID": {}
        },
        {
          "ID": 124,
          "Name": "TPS+",
          "ParentID": 123
        },
        {
          "ID": 125,
          "Name": "TPS Qatar",
          "ParentID": 124
        }
      ]
    },
    {
      "name": "AnalyticalDim3",
      "alias": "AnalyticalDim3",
      "order": 9,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 37,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 38,
          "Name": "UMSQ",
          "ParentID": 37
        },
        {
          "ID": 39,
          "Name": "HQW2",
          "ParentID": 37
        },
        {
          "ID": 40,
          "Name": "PRO-MAIN",
          "ParentID": 37
        },
        {
          "ID": 41,
          "Name": "SHJ",
          "ParentID": 37
        },
        {
          "ID": 42,
          "Name": "DOH1",
          "ParentID": 37
        },
        {
          "ID": 43,
          "Name": "AL QUOZ",
          "ParentID": 37
        },
        {
          "ID": 44,
          "Name": "CAFE",
          "ParentID": 37
        },
        {
          "ID": 45,
          "Name": "MRI",
          "ParentID": 37
        },
        {
          "ID": 46,
          "Name": "x",
          "ParentID": 37
        },
        {
          "ID": 47,
          "Name": "DAMAGE",
          "ParentID": 37
        },
        {
          "ID": 48,
          "Name": "DSO",
          "ParentID": 37
        },
        {
          "ID": 49,
          "Name": "JVC",
          "ParentID": 37
        },
        {
          "ID": 50,
          "Name": "DLM",
          "ParentID": 37
        },
        {
          "ID": 51,
          "Name": "FZN",
          "ParentID": 37
        },
        {
          "ID": 52,
          "Name": "REM",
          "ParentID": 37
        },
        {
          "ID": 53,
          "Name": "HQWCD",
          "ParentID": 37
        },
        {
          "ID": 54,
          "Name": "QUAR",
          "ParentID": 37
        },
        {
          "ID": 55,
          "Name": "HO",
          "ParentID": 37
        },
        {
          "ID": 56,
          "Name": "CREEK",
          "ParentID": 37
        },
        {
          "ID": 57,
          "Name": "GRM",
          "ParentID": 37
        },
        {
          "ID": 58,
          "Name": "IMP-SHORT",
          "ParentID": 37
        },
        {
          "ID": 59,
          "Name": "MRK",
          "ParentID": 37
        },
        {
          "ID": 60,
          "Name": "DIC",
          "ParentID": 37
        },
        {
          "ID": 61,
          "Name": "PPLUS",
          "ParentID": 37
        },
        {
          "ID": 62,
          "Name": "HQW",
          "ParentID": 37
        },
        {
          "ID": 63,
          "Name": "SZR",
          "ParentID": 37
        },
        {
          "ID": 64,
          "Name": "MEAISEM",
          "ParentID": 37
        },
        {
          "ID": 65,
          "Name": "PRL",
          "ParentID": 37
        },
        {
          "ID": 66,
          "Name": "RAK",
          "ParentID": 37
        },
        {
          "ID": 67,
          "Name": "VILLA AJ",
          "ParentID": 37
        },
        {
          "ID": 68,
          "Name": "DIP",
          "ParentID": 37
        },
        {
          "ID": 69,
          "Name": "MTR/ALK",
          "ParentID": 37
        },
        {
          "ID": 70,
          "Name": "PALM",
          "ParentID": 37
        },
        {
          "ID": 71,
          "Name": "WSL",
          "ParentID": 37
        },
        {
          "ID": 72,
          "Name": "MOBILE",
          "ParentID": 37
        },
        {
          "ID": 73,
          "Name": "No Location",
          "ParentID": 37
        },
        {
          "ID": 74,
          "Name": "XX2",
          "ParentID": 37
        },
        {
          "ID": 75,
          "Name": "XX1",
          "ParentID": 37
        },
        {
          "ID": 76,
          "Name": "XX3",
          "ParentID": 37
        }
      ]
    },
    {
      "name": "AnalyticalDim4",
      "alias": "Normilization",
      "order": 11,
      "default": "Top",
      "visible": False,
      "dimensionContent": [
        {
          "ID": 40,
          "Name": "Top",
          "ParentID": {}
        },
        {
          "ID": 41,
          "Name": "Normilization",
          "ParentID": 40
        },
        {
          "ID": 42,
          "Name": "Pre-OPEX",
          "ParentID": 40
        },
        {
          "ID": 43,
          "Name": "Accounting Treat - Visa Prepaid Write-off ",
          "ParentID": 41
        },
        {
          "ID": 44,
          "Name": "Leave Provision - Accounting Adjustment",
          "ParentID": 41
        },
        {
          "ID": 45,
          "Name": "Legal/Court Case",
          "ParentID": 41
        },
        {
          "ID": 46,
          "Name": "Lucky Vet - Acquisition Expenses",
          "ParentID": 41
        },
        {
          "ID": 47,
          "Name": "Nishe Services for VAT and Corp. Tax",
          "ParentID": 41
        },
        {
          "ID": 48,
          "Name": "Tech Recruitment",
          "ParentID": 41
        },
        {
          "ID": 49,
          "Name": "Pre-OPEX - Petsville DIP",
          "ParentID": 42
        },
        {
          "ID": 50,
          "Name": "Pre-OPEX - Silicon Oasis",
          "ParentID": 42
        },
        {
          "ID": 51,
          "Name": "Pre-OPEX Creek",
          "ParentID": 42
        },
        {
          "ID": 52,
          "Name": "Pre-OPEX Dubai Marina",
          "ParentID": 42
        },
        {
          "ID": 53,
          "Name": "Pre-OPEX Reem",
          "ParentID": 42
        },
        {
          "ID": 54,
          "Name": "Pre-OPEX Reem (Store Opening)",
          "ParentID": 42
        },
        {
          "ID": 55,
          "Name": "Pre-OPEX Umm Suqeim",
          "ParentID": 42
        },
        {
          "ID": 56,
          "Name": "Pre-Opex Warehouse",
          "ParentID": 42
        },
        {
          "ID": 57,
          "Name": "Pre-Opex Warehouse (PV)",
          "ParentID": 42
        },
        {
          "ID": 59,
          "Name": "No Normalization",
          "ParentID": 40
        }
      ]
    }
  ]
}


resultstate = create_dynamic_list(state)
print (resultstate)

{'JsonLists': [{'12periods': {'dimensions': ['Year', 'Period'], 'type': 'dynamic', 'dynamicconfig': {'relationship': 'rolling', 'startPeriod': {'Year': '', 'Period': ''}, 'length': 12, 'direction': 'backward'}}}]}
